## Combining all S&P 500 company prices into one DataFrame

https://pythonprogramming.net/combining-stock-prices-into-one-dataframe-python-programming-for-finance/

In [1]:
import bs4 as bs
import datetime as dt
import os
import yfinance as yf
import pickle
import requests


def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker.strip())
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers

# save_sp500_tickers()
def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')

    start = dt.datetime(2010, 1, 1)
    end = dt.datetime.now()
    for ticker in tickers:
        # e.g. BRK.B => BRK-B, BF.B => BF-B
        ticker = ticker.replace('.', '-')
        # just in case your connection breaks, we'd like to save our progress!
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = yf.download(ticker, start=start, end=end)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))


# get_data_from_yahoo()

While we do have all of the data at our disposal, we may want to actually assess the data together. To do this, we're going to join all of the stock datasets together. Each of the stock files at the moment come with: `Open`, `High`, `Low`, `Close`, `Volume`, and `Adj Close`. At least to start, we're mostly just interested in the adjusted close for now.

In [2]:
# def compile_data():
#     with open("sp500tickers.pickle","rb") as f:
#         tickers = pickle.load(f)

#     main_df = pd.DataFrame()

To begin, we pull our previously-made list of tickers, and begin with an empty DataFrame, called main_df. Now, we're ready to read in each stock's dataframe:

In [3]:
#     for count,ticker in enumerate(tickers):
#         df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
#         df.set_index('Date', inplace=True)

You do not need to use [Python's enumerate](https://pythonprogramming.net/enumerate-intermediate-python-tutorial/) here, I am just using it so we know where we are in the process of reading in all of the data. You could just iterate over the tickers. From this point, we **could** generate extra columns with interesting data, like:

In [4]:
#         df['{}_HL_pct_diff'.format(ticker)] = (df['High'] - df['Low']) / df['Low']
#         df['{}_daily_pct_chng'.format(ticker)] = (df['Close'] - df['Open']) / df['Open']

For now, however, we're not going to be bothered with this. Just know this could be a path to pursue down the road. Instead, we're really just interested in that `Adj Close` column:

In [5]:
#         df.rename(columns={'Adj Close':ticker}, inplace=True)
#         df.drop(['Open','High','Low','Close','Volume'],1,inplace=True)

Now we've got just that column (or maybe extras, like above...but remember, in this example, we're not doing the `HL_pct_diff` or `daily_pct_chng`). Notice that we have renamed the `Adj Close` column to whatever the ticker name is. Let's begin building the shared dataframe:

In [6]:
#         if main_df.empty:
#             main_df = df
#         else:
#             main_df = main_df.join(df, how='outer')

If there's nothing in the main_df, then we'll start with the current df, otherwise we're going to use [Pandas' join](https://pythonprogramming.net/join-merge-data-analysis-python-pandas-tutorial/).

Still within this for loop, we'll add two more lines:

In [7]:
#         if count % 10 == 0:
#             print(count)

This will just output the count of the current ticker if it's evenly divisible by 10. What `count % 10` gives us is the remainder if count was to be divided by 10. So if we ask `if count % 10 == 0`, we're only going to see the if statement True if the current count, divided by 10, has a remainder of 0, or if it is perfectly divisible by 10.

When we're all done with the for-loop:

In [8]:
#     print(main_df.head())
#     main_df.to_csv('sp500_joined_closes.csv')

This function and calling it up to this point:

In [9]:
#     with open("sp500tickers.pickle","rb") as f:
#         tickers = pickle.load(f)

#     main_df = pd.DataFrame()

#     for count,ticker in enumerate(tickers):
#         df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
#         df.set_index('Date', inplace=True)

#         df.rename(columns={'Adj Close':ticker}, inplace=True)
#         df.drop(['Open','High','Low','Close','Volume'],1,inplace=True)

#         if main_df.empty:
#             main_df = df
#         else:
#             main_df = main_df.join(df, how='outer')

#         if count % 10 == 0:
#             print(count)
#     print(main_df.head())
#     main_df.to_csv('sp500_joined_closes.csv')


# compile_data()

Full code up to this point:

In [10]:
%%time
import bs4 as bs
import datetime as dt
import os
import pandas as pd
import yfinance as yf
import pickle
import requests
from tqdm import tqdm
from IPython.display import display


def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker.strip())
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers


def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')

    start = dt.datetime(2010, 1, 1)
    end = dt.datetime.now()
    for ticker in tickers:
        # e.g. BRK.B => BRK-B, BF.B => BF-B
        ticker = ticker.replace('.', '-')
        # just in case your connection breaks, we'd like to save our progress!
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = yf.download(ticker, start=start, end=end)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))


def compile_data():
    with open("sp500tickers.pickle", "rb") as f:
        tickers = pickle.load(f)

    main_df = pd.DataFrame()

    for ticker in tqdm(tickers):
        # e.g. BRK.B => BRK-B, BF.B => BF-B
        ticker = ticker.replace('.', '-')
        df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
        df.set_index('Date', inplace=True)

        df.rename(columns={'Adj Close': ticker}, inplace=True)
        df.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'], inplace=True)

        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how='outer')

    display(main_df)
    main_df.to_csv('sp500_joined_closes.csv')


compile_data()

100%|████████████████████████████████████████████████████████████████| 503/503 [00:02<00:00, 193.32it/s]


,MMM,AOS,ABT,ABBV,ACN,ADM,ADBE,ADP,AES,AFL,...,WTW,GWW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,55.033230,6.061760,19.138166,NaN,32.455265,22.142736,37.090000,27.043221,9.865004,16.935186,...,53.240776,76.363777,41.758171,13.167749,NaN,19.345045,28.670000,52.791035,10.726543,NaN
2010-01-05,54.688534,5.984308,18.983543,NaN,32.655857,22.262346,37.700001,26.897989,9.763973,17.426476,...,53.121429,76.395195,44.297863,13.011587,NaN,19.278898,28.620001,54.462196,11.104751,NaN
2010-01-06,55.464123,5.987023,19.088966,NaN,33.002991,22.206062,37.619999,26.834860,9.662939,17.579561,...,53.976639,76.685776,43.716812,13.036573,NaN,19.141064,28.400000,54.444588,12.070378,NaN
2010-01-07,55.503906,6.004689,19.247101,NaN,32.972153,21.973869,36.889999,26.822224,9.670161,17.768244,...,53.817532,77.408264,44.650425,12.980354,NaN,19.135551,27.690001,55.693562,13.422259,NaN
2010-01-08,55.894970,6.093013,19.345501,NaN,32.841000,21.699457,36.689999,26.784344,9.944386,17.590239,...,53.757862,78.319237,44.330513,12.986597,NaN,19.141064,27.600000,54.523750,13.204995,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,90.120003,66.639999,94.809998,146.309998,296.089996,69.470001,539.559998,240.449997,14.530000,77.760002,...,207.889999,686.710022,91.419998,59.080002,89.440002,119.910004,205.910004,104.839996,29.860001,167.119995
2023-10-25,89.389999,65.190002,93.570000,145.259995,292.679993,71.050003,521.140015,218.330002,14.680000,78.260002,...,207.740005,677.409973,88.940002,59.470001,87.760002,120.309998,198.910004,103.639999,29.629999,163.669998
2023-10-26,88.650002,69.120003,93.980003,145.199997,292.040009,71.849998,514.280029,218.839996,14.900000,77.519997,...,229.080002,699.669983,87.540001,59.770000,88.690002,118.750000,204.830002,103.120003,30.450001,158.070007


CPU times: user 3.32 s, sys: 366 ms, total: 3.68 s
Wall time: 3.69 s


In the next tutorial, we're going to attempt to see if we can quickly find any relationships in the data.